In [11]:
using MLDatasets, Statistics

include("ConvolutionModule.jl")  # Load the module
include("PoolingModule.jl")  # Load the module
using .ConvolutionModule, .PoolingModule # Use the namespace

train_x, train_y = MNIST(split=:train)[:]
input_image = reshape(train_x[:, :, 1], 28, 28, 1)
input_image = Float64.(input_image)

# Initialize the convolutional layer
conv_layer = ConvolutionModule.init_conv_layer(3, 3, 1, 6, 1, 1)

# Initialize the max pooling layer
pool_layer = PoolingModule.init_pool_layer(2, 2, 2)  # 2x2 pool size, stride of 2



# Apply the convolutional layer
conv_output = conv_layer(input_image)

# Apply the max pooling layer
pooled_output = PoolingModule.apply_pooling(pool_layer, conv_output)


UndefVarError: UndefVarError: `conv_output` not defined